In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('http://www.ppgia.pucpr.br/~jean.barddal/datascience/enron.csv')

In [3]:
df.head(5)

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name
0,4175000.0,2869717.0,-3081055.0,NaN,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,NaN,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0,ALLEN PHILLIP K
1,NaN,178980.0,NaN,NaN,NaN,257817.0,3486.0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466.0,257817.0,BADUM JAMES P
2,NaN,NaN,-5104.0,NaN,james.bannantine@enron.com,4046157.0,56301.0,29.0,39.0,0.0,NaN,NaN,864523.0,False,1757552.0,-560222.0,477.0,465.0,566.0,916197.0,5243487.0,BANNANTINE JAMES M
3,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,NaN,NaN,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0,BAXTER JOHN C
4,400000.0,260455.0,-201641.0,NaN,frank.bay@enron.com,NaN,129142.0,NaN,NaN,NaN,NaN,NaN,69.0,False,145796.0,-82782.0,239671.0,NaN,NaN,827696.0,63014.0,BAY FRANKLIN R


In [4]:
# implementar o metodo de tukey

# lista de outliers (outliers para mais de n variaveis)
outliers = []
n = 2

for col in df.select_dtypes(include='number').columns.values:
  print(col)

  # calcular os quartis (Q3 e Q1)
  q1 = df[col].quantile(q=0.25)
  q3 = df[col].quantile(q=0.75)

  # calcular o tamanho da caixa (IQR = Q3 - Q1)
  iqr = q3 - q1

  # limites superiores e inferiores (whiskers/bigodes)
  sup = q3 + 1.5 * iqr 
  inf = q1 - 1.5 * iqr

  # obtendo os dados abaixo ou acima dos whiskers
  ix_outliers_col = list(df[(df[col] < inf) | (df[col] > sup)].index)

  print(ix_outliers_col)
  # armazenando os outliers dessa coluna na lista global de outliers
  for v in ix_outliers_col:
    outliers.append(v)    

bonus
[0, 7, 31, 42, 75, 78, 79, 91, 122, 130, 139]
deferral_payments
[0, 47, 66, 68, 130]
deferred_income
[0, 7, 47, 59, 112, 130]
director_fees
[8, 11, 94, 108, 130, 132]
exercised_stock_options
[3, 32, 35, 47, 65, 79, 103, 110, 112, 122, 130, 144]
expenses
[88, 116, 130, 132]
from_messages
[0, 6, 7, 19, 31, 32, 58, 62, 66, 72, 73, 75, 78, 89, 116, 117, 139]
from_poi_to_this_person
[7, 20, 26, 34, 37, 47, 58, 75, 78, 110, 139]
from_this_person_to_poi
[0, 6, 7, 19, 31, 58, 72, 73, 75, 78, 89, 116, 117]
loan_advances
[]
long_term_incentive
[40, 78, 79, 86, 130]
other
[3, 47, 69, 77, 79, 86, 103, 119, 121, 130, 140]
restricted_stock
[3, 11, 22, 47, 60, 69, 73, 79, 103, 112, 122, 130, 139, 140, 144]
restricted_stock_deferred
[11, 32, 130]
salary
[2, 32, 47, 57, 79, 105, 122, 130, 139]
shared_receipt_with_poi
[7, 117]
to_messages
[6, 7, 73, 75, 78, 117, 139]
total_payments
[3, 7, 11, 31, 47, 78, 79, 86, 122, 130, 139]
total_stock_value
[3, 32, 35, 41, 47, 65, 66, 79, 102, 103, 110, 112, 1

In [5]:
from collections import Counter
lista_indices = [x[0] for x in Counter(outliers).most_common(10)]
lista_indices

[130, 47, 7, 79, 78, 139, 122, 0, 75, 3]

In [6]:
Counter(outliers).most_common(100)

[(130, 13),
 (47, 9),
 (7, 8),
 (79, 8),
 (78, 7),
 (139, 7),
 (122, 6),
 (0, 5),
 (75, 5),
 (3, 5),
 (32, 5),
 (31, 4),
 (112, 4),
 (11, 4),
 (103, 4),
 (73, 4),
 (117, 4),
 (66, 3),
 (110, 3),
 (144, 3),
 (116, 3),
 (6, 3),
 (58, 3),
 (86, 3),
 (140, 3),
 (132, 2),
 (35, 2),
 (65, 2),
 (19, 2),
 (72, 2),
 (89, 2),
 (69, 2),
 (42, 1),
 (91, 1),
 (68, 1),
 (59, 1),
 (8, 1),
 (94, 1),
 (108, 1),
 (88, 1),
 (62, 1),
 (20, 1),
 (26, 1),
 (34, 1),
 (37, 1),
 (40, 1),
 (77, 1),
 (119, 1),
 (121, 1),
 (22, 1),
 (60, 1),
 (2, 1),
 (57, 1),
 (105, 1),
 (41, 1),
 (102, 1)]

In [7]:
df.loc[lista_indices]

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name
130,97343619.0,32083396.0,-27992891.0,1398517.0,NaN,311764000.0,5235198.0,NaN,NaN,NaN,83925000.0,48521928.0,42667589.0,False,130322299.0,-7576788.0,26704229.0,NaN,NaN,309886585.0,434509511.0,TOTAL
47,2000000.0,6426990.0,-3367011.0,NaN,mark.frevert@enron.com,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,False,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0,FREVERT MARK A
7,5249999.0,2144013.0,-2334434.0,NaN,tim.belden@enron.com,953136.0,17355.0,484.0,228.0,108.0,NaN,NaN,210698.0,True,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0,BELDEN TIMOTHY N
79,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0,LAY KENNETH L
78,8000000.0,NaN,NaN,NaN,john.lavorato@enron.com,4158995.0,49537.0,2585.0,528.0,411.0,NaN,2035380.0,1552.0,False,1008149.0,NaN,339288.0,3962.0,7259.0,10425757.0,5167144.0,LAVORATO JOHN J
139,3000000.0,NaN,NaN,NaN,greg.whalley@enron.com,3282960.0,57838.0,556.0,186.0,24.0,NaN,808346.0,301026.0,False,2796177.0,NaN,510364.0,3920.0,6019.0,4677574.0,6079137.0,WHALLEY LAWRENCE G
122,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0,SKILLING JEFFREY K
0,4175000.0,2869717.0,-3081055.0,NaN,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,NaN,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0,ALLEN PHILLIP K
75,3100000.0,NaN,NaN,NaN,louise.kitchen@enron.com,81042.0,5774.0,1728.0,251.0,194.0,NaN,NaN,93925.0,False,466101.0,NaN,271442.0,3669.0,8305.0,3471141.0,547143.0,KITCHEN LOUISE
3,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,NaN,NaN,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0,BAXTER JOHN C


#### Isolation Forest

In [8]:
from sklearn.ensemble import IsolationForest

In [10]:
# obtendo apenas colunas numericas
df_num = df.select_dtypes(include='number')

In [11]:
# imputando de valores faltantes (substituir valores faltantes por 0)
df_num.fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [12]:
# criar a isolation forest
isof = IsolationForest(random_state=51)
isof.fit(df_num)

IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=51, verbose=0, warm_start=False)

In [13]:
# obtendo marcacoes de outliers
y_pred = isof.predict(df_num)

In [15]:
# adicionando coluna nova na tabela com a marcacao
df['inlier'] = y_pred

In [18]:
# filtrando apenas os outliers
df[df['inlier'] == -1]

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name,inlier
7,5249999.0,2144013.0,-2334434.0,NaN,tim.belden@enron.com,953136.0,17355.0,484.0,228.0,108.0,NaN,NaN,210698.0,True,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0,BELDEN TIMOTHY N,-1
11,NaN,NaN,NaN,137864.0,sanjay.bhatnagar@enron.com,2604490.0,NaN,29.0,0.0,1.0,NaN,NaN,137864.0,False,-2604490.0,15456290.0,NaN,463.0,523.0,15456290.0,NaN,BHATNAGAR SANJAY,-1
47,2000000.0,6426990.0,-3367011.0,NaN,mark.frevert@enron.com,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,False,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0,FREVERT MARK A,-1
73,1000000.0,NaN,NaN,NaN,steven.kean@enron.com,2022048.0,41953.0,6759.0,140.0,387.0,NaN,300000.0,1231.0,False,4131594.0,NaN,404338.0,3639.0,12754.0,1747522.0,6153642.0,KEAN STEVEN J,-1
78,8000000.0,NaN,NaN,NaN,john.lavorato@enron.com,4158995.0,49537.0,2585.0,528.0,411.0,NaN,2035380.0,1552.0,False,1008149.0,NaN,339288.0,3962.0,7259.0,10425757.0,5167144.0,LAVORATO JOHN J,-1
79,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0,LAY KENNETH L,-1
122,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0,SKILLING JEFFREY K,-1
130,97343619.0,32083396.0,-27992891.0,1398517.0,NaN,311764000.0,5235198.0,NaN,NaN,NaN,83925000.0,48521928.0,42667589.0,False,130322299.0,-7576788.0,26704229.0,NaN,NaN,309886585.0,434509511.0,TOTAL,-1


In [20]:
# quao menor o score, mais outlier!
scores = isof.score_samples(df_num)
df['score'] = scores

In [22]:
df[df['inlier'] == -1]

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name,inlier,score
7,5249999.0,2144013.0,-2334434.0,NaN,tim.belden@enron.com,953136.0,17355.0,484.0,228.0,108.0,NaN,NaN,210698.0,True,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0,BELDEN TIMOTHY N,-1,-0.540658
11,NaN,NaN,NaN,137864.0,sanjay.bhatnagar@enron.com,2604490.0,NaN,29.0,0.0,1.0,NaN,NaN,137864.0,False,-2604490.0,15456290.0,NaN,463.0,523.0,15456290.0,NaN,BHATNAGAR SANJAY,-1,-0.557951
47,2000000.0,6426990.0,-3367011.0,NaN,mark.frevert@enron.com,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,False,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0,FREVERT MARK A,-1,-0.632228
73,1000000.0,NaN,NaN,NaN,steven.kean@enron.com,2022048.0,41953.0,6759.0,140.0,387.0,NaN,300000.0,1231.0,False,4131594.0,NaN,404338.0,3639.0,12754.0,1747522.0,6153642.0,KEAN STEVEN J,-1,-0.515462
78,8000000.0,NaN,NaN,NaN,john.lavorato@enron.com,4158995.0,49537.0,2585.0,528.0,411.0,NaN,2035380.0,1552.0,False,1008149.0,NaN,339288.0,3962.0,7259.0,10425757.0,5167144.0,LAVORATO JOHN J,-1,-0.571072
79,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0,LAY KENNETH L,-1,-0.719847
122,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0,SKILLING JEFFREY K,-1,-0.531281
130,97343619.0,32083396.0,-27992891.0,1398517.0,NaN,311764000.0,5235198.0,NaN,NaN,NaN,83925000.0,48521928.0,42667589.0,False,130322299.0,-7576788.0,26704229.0,NaN,NaN,309886585.0,434509511.0,TOTAL,-1,-0.886864
